# Pre Lab 08 : System of Linear Equations

## Objectives

The main objectives of this lab are as follows.

- Learn to solve systems of linear equations.
- Prepare for the lab. **The prelab is essential for completing the lab**. In the lab this week we will solve the heat equation in one dimension. This can be confusing if we try to jump in without working through the set up of the problem. The prelab will lead us through the set up making the lab much easier (and faster). **It is in your best interest to understand this setup prior to the lab.**

## Initialization

As always you should add initialization to the top of your notebook.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## System of Linear Equations

Every system of linear equations does not have a solution. Here we will study a particular case and see how problems show up. (There is another example of this in the homework.)

Consider the system of equations
\begin{align}
x + y + z &= 1, \\
x + 2y + 4z &= \eta, \\
x + 4y + 10z &= \eta^2.
\end{align}
This system of equations only has solutions for specific values of $\eta$.

**Analytically** find the values of $\eta$ for which the system of equations given above has a solution. You should find two such values. Provide those two values below showing how you found them.
(*Hint:* Consider reducing the system of equations by subtracting the first from the second and by subtracting the second from the third. The two remaining equations will then provide a single equation that $\eta$ must satisfy.)

YOUR ANSWER HERE

**Analytically** solve the system of equations, finding a point ($x$, $y$, $z$) that satisfies all the equations for each of the values of $\eta$. There are "trivial" solutions that can be found by just looking at the system of equations. Provide the solutions below. (*Hint:* Zero is your friend, use it liberally. It has not been said that there is a unique solution.)

YOUR ANSWER HERE

You may have noticed that the solutions to this system of equations for the two values of $\eta$ are not unique; in other words there is not a single point ($x$, $y$, $z$) that satisfies the equation. This causes problems when solving the system $\mathsf{A}\vec x=\vec b$ using `scipy.linalg.solve`.

Construct the matrix `A` and the vector `b` as NumPy arrays. Use these with `scipy.linalg.solve` to try to solve the system of equations. You may get a warning or an error (or perhaps even should), this is expected.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Recall that we said that there would be a unique solution to our system of equations if the determinant of our matrix is non-zero. Verify that the matrix you created is in fact singular with the help of `scipy.linalg.det`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Heat Equation

In the lab this week we will solve the heat equation in one dimension.

Let $T(t,x)$ represent the temperature at time $t$ and position $x$ for some object. It satisfies the partial differential equation

$$ \frac{\partial T(t,x)}{\partial t} = \frac{\partial^2 T(t,x)}{\partial x^2}. $$

We have not talked about solving partial differential equations and will not do so here. Instead, we will turn this partial differential equation into a **system of linear equations** that can be solved at each time step. We know the basics of how to do this: discretize the derivatives and solve a difference equation. Just as we saw with ordinary differential equations, there are many details with finding a good algorithm for solving a partial differential equation. Issues such as stability, convergence, step sizes, *etc.* must be considered for each type of partial differential equation we wish to solve. In this case we will use the *Crank-Nicolson* method. This is an implicit method that is unconditionally stable and second order accurate. The result of applying this method is to convert the original differential equation into the **system of linear equations**
$$
-r T(t_{j+1},x_{i-1}) + (1+2r) T(t_{j+1},x_i) - r T(t_{j+1},x_{i+1}) = r T(t_j,x_{i-1}) + (1-2r) T(t_j,x_i) +r T(t_j,x_{i+1}),
$$
where
$$ r\equiv \frac{\Delta t}{2(\Delta x)^2},$$
and $\Delta t$ and $\Delta x$ are the time and space step sizes, respectively, as will be discussed in the lab. We will use this system of linear equations as the starting point for our study.

### Understanding the Linear System

Converting the partial differential equation to a linear system of equations is nice, but we still need to understand this linear system. Notice that our system of equations can be written in the form
$$ \mathsf{A}\vec T_{j+1} = \mathsf{B} \vec T_j \equiv \vec \tau_j. $$
Here $\vec T_j$ represents the temperatures at time $t_j$ for the positions $x_0$, $x_1$, $\ldots$, $x_N$; in other words, $\vec T_j = T(t_j, \vec{x}) = (T(t_j, x_0)$, $T(t_j, x_1)$, $\ldots$, $T(t_j, x_N))$. Also, the matrices $\mathsf{A}$ and $\mathsf{B}$ can be written in terms of known quantities. Thus, if we know the temperatures at time, $t_j$, then the right hand side of the equation, $\vec\tau_j$, is also known so that we can determine $\vec T_{j+1}$ by just solving the linear system of equations as we have discussed in class!

To proceed it is best to grab some scratch paper as we will first consider a small system where we can explicitly write out all the equations, then generalize the result in the lab to a larger system.

In the lab we will evolve the temperature along a rod of length $L=1$ for various initial conditions, $T(0,x)$, and will also specify particular boundary conditions (the behavior of $T(t,0)$ and $T(t,L)$). Given the initial conditions means that $\vec T_0$ is a **known vector**. Suppose that we divide our rod into 4 segments at the points $\{ x_0, x_1, x_2, x_3, x_4 \}$ so that $N=4$. (In the lab these points will be equally spaced, however, it is best to derive results in a generic manner and specialize when needed.) Starting from the initial conditions means starting from the initial time step, $j=0$, so that the right hand side of our system of equations is known. With this, write out the complete **system of linear equations**. Do this for yourself on your scratch paper, we will use these results shortly. You should see some patterns in the equations which we will exploit to more efficiently solve them. In doing this you should find you need quantities like $T(t,x_{-1})$ and $T(t,x_{N+1})$. In principle the values of these will depend on our choice of boundary conditions. Here we will just set them equal to zero.

#### The matrix $\mathsf{A}$

We first identify the matrix $\mathsf{A}$. On the left hand side of the system of equations you just wrote out, the coefficients of the $T(t_1,x_i)$ terms are the elements of the matrix $\mathsf{A}$. This should be a $5\times 5$ matrix which we can now construct. Furthermore, it should have a "simple" structure; it should be a **tridiagonal matrix**. That is, the only non-zero elements should lie along the main diagonal and along the diagonal lines one above and one below the main diagonal.

Write the matrix $\mathsf{A}$ below. To help, the array has been written below with the elements of the matrix set as dots ('.') in the structure. Copy the LaTeX code to the solution cell. If you do not change any of the other structure (all the ampersands, for example) you will end up with a nicely formatted matrix.

$$ \mathsf{A} = \begin{pmatrix}
. & . & . & . & . \\
. & . & . & . & . \\
. & . & . & . & . \\
. & . & . & . & . \\
. & . & . & . & . 
\end{pmatrix}$$

YOUR ANSWER HERE

#### The vector $\vec\tau_j$

We could handle the right hand side in a similar way by writing out the matrix $\mathsf{B}$ and multiplying it with the vector $\vec T_j$, however, **it is more efficient to not do this**. You should see that $\mathsf{B}$ will also be a tridiagonal matrix, thus there will be many zeros in it (particularly when we use a larger $N$). Due to this it is better to write a function that, given $\vec T_j$ and $r$, returns the vector $\vec\tau_j$ **without constructing nor multiplying by the matrix $\vec{B}$**.

Write a function that is given the known vector $\vec T_j$ and a number $r$ and returns the vector $\vec\tau_j$. Note that this can be done **without using loops** and for arbitrary (and unspecified) $N$ by smart use of array slicing. You should write your function in this way.

In [ ]:
def tau(T, r):
# YOUR CODE HERE
raise NotImplementedError()

A very simple test case is when $r=0$. This is not a realistic case, but can be used as a first test. In this case $\tau(t_j,x_i) = T(t_j,x_i)$.

In [ ]:
T_test = np.random.random(size=5)
r_test = 0.0
assert(np.allclose(tau(T_test,r_test), T_test))


To test your function more thoroughly, consider the case of a constant $\vec T_j$ (for example all ones) and choose some value for $r$ in the system of equations you wrote out. This is a simple case which does not catch all possible errors but has an easy to determine result. (*Hint*: It may help to explicitly write out the elements of $\vec\tau_j$. Using the above equations: $\tau(t_j, x_i) =  r T(t_j,x_{i-1}) + (1-2r) T(t_j,x_i) +r T(t_j,x_{i+1})$.)

Verify your function by constructing and showing it passes a simple test case.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Solving the Linear System

Now that we know how to represent the matrix $\mathsf{A}$ and have written code to calculate the vector $\vec\tau_j$ we can start at time $t_0$ with a known $\vec T_0$ and calculate $\vec T_1$ by solving the linear system of equations to find the temperatures along the rod $\vec T_1$ at time $t_1$. We could do this using `scipy.linalg.solve`, however, that is not very efficient. As we have seen, our matrix $\mathsf{A}$ is tridiagonal and we can use that to both speed up the solution and to simplify the code.  Notice that we have not yet written any code to construct the matrix $\mathsf{A}$. Although there are clever ways of doing this, we can actually avoid the issue by storing the information in a much simpler, more efficient manner. To exploit the structure of $\mathsf{A}$ we will instead use `scipy.linalg.solve_banded`. Look up its documentation!  We next need to understand how to use it.

The documentation fully describes how to store the matrix in a form that exploits its banded nature. The function works for an arbitrary banded matrix. Although the documentation is good, to further help let us write out the example they discuss in a bit more detail. The example in the documentation is a $6\times 6$ banded matrix that we can write out in its full glory as

$$\mathsf{a} = \begin{pmatrix}
a_{00} & a_{01} & 0 & 0 & 0 & 0 \\
a_{10} & a_{11} & a_{12} & 0 & 0 & 0 \\
a_{20} & a_{21} & a_{22} & a_{23} & 0 & 0 \\
0 & a_{31} & a_{32} & a_{33} & a_{34} & 0 \\
0 & 0 & a_{42} & a_{43} & a_{44} & a_{45} \\
0 & 0 & 0 & a_{53} & a_{54} & a_{55}
\end{pmatrix}.$$

This is converted to the banded version, `ab`, as shown in the documentation. In `ab` the elements marked by an asterisk (`*`) are not used by the algorithm so may be set to any value. *The values of `*` quantities are ignored.* Finally, the algorithm needs to know the number of "bands" below and above the main diagonal. These are specified by the `tuple` `(l,u)`. Study the documentation to make sure you understand how this matrix, $\mathrm{a}$, gets converted to the banded version `ab` and what the values `l` and `u` represent.

For our tridiagonal matrix, $\mathsf{A}$, what are the values for `l` and `u`?

YOUR ANSWER HERE

For our tridiagonal matrix, $\mathrm{A}$, write a function to construct its banded version `ab` as used by `scipy.linalg.sovle_banded`. Here `N` is the size of the matrix and `r` is the $r$ parameter defined above. Provide the code below. Also print the matrix for the case when `N=5` and `r=0.25`.

This should be *very* simple. If you find yourself writing a lot of code or using a loop then you are making your life difficult, there *is* an easier way.

In [ ]:
def coefficient_matrix(N, r):
# YOUR CODE HERE
raise NotImplementedError()

Note that our matrix $\mathsf{A}$ is actually a symmetric tridiagonal matrix. Exploiting this extra symmetry would be even more efficient in terms of memory use and computational time. This could be done using the even more specialized function `scipy.linalg.solveh_banded`. Feel free to look up this function and how to use it, though this will not be required for the problems in the lab, you can use it if you desire.

## Turning in the PreLab

All prelabs will be handled as was done for PreLab01. See that file for details. It will be assumed from now on that you have read and understood the procedure and what it means when you submit a prelab.